## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [36]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
from nltk.stem import SnowballStemmer


data = pd.read_csv('/datasets/data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [37]:
data.describe() #сводная статистика

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [38]:
data.info() # Отсутствуют значения в столбцах days_emploed  и total_income

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [39]:
print("Люди с нулевым возрастом: {}".format(data[data['dob_years'] == 0].count()[0]))
print("Семьи с -1 ребенком: {}".format(data[data['children'] == -1].count()[0]))
print("Семьи с 20 детьми: {}".format(data[data['children'] == 20].count()[0]))

Люди с нулевым возрастом: 101
Семьи с -1 ребенком: 47
Семьи с 20 детьми: 76


### Вывод

1. Загружаем библиотеку Pandas
2. Считываем данные из файла /datasets/data.csv
3. Предварительный просмотр данных
  Вывод по сводной таблице:
1. Максимальное количество детей 20(удивительно, но возможно. Скорее всего ошибка), минимальное -1(странно, но скорее всего ошибка)
2. Трудовой стаж не может быть отрицательным, поэтому возьмем столбец по модулю, чтобы убрать недоразумение.
3. -1 ребенок и 20 детей скорее всего это 1 и 2 ребенка соответственно. Просто ошибка при заполненинии анкеты.


### Шаг 2. Предобработка данных

### Обработка пропусков

In [40]:
data['total_income'] = data['total_income'].abs()
data['days_employed'] = data['days_employed'].abs()
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,66914.728907,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,139030.880527,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,927.009265,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,2194.220567,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,5537.882441,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [41]:
data['days_employed'] = data.groupby('income_type')['days_employed'].transform(lambda x: x.fillna(x.mean()*data['dob_years']*365))
data['total_income'] = data.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.mean()))
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [43]:
data['children'] = data['children'].abs() # Легким движением руки преобразуем количество детей из -1 в 1 :)
data['children'] = data['children'].replace(20, 2) # Точно так же уберем часть многодетных семей

In [44]:
data['education'] = data['education'].str.lower() #Приведем образование к единому размеру.
print("Количество строк с нулевым возрастом = {}".format(data[data['dob_years'] == 0].count()[0]))
print("Количество строк с несовершеннолетними = {}".format(data[data['dob_years'] < 19].count()[0]))

Количество строк с нулевым возрастом = 101
Количество строк с несовершеннолетними = 101


In [45]:
#Количество одинаковое. Заполним эти поля средним значением по столбцу.

In [46]:
data['dob_years'] = data.groupby('income_type')['dob_years'].transform(lambda x: x.replace(0, int(x.mean())))
print("Количество строк с нулевым значемнием = {}".format(data[data['dob_years'] == 0].count()[0]))

Количество строк с нулевым значемнием = 0


In [47]:
data['days_employed'] = data['days_employed'].astype('int') # Округлим до целого для удобства
data['total_income'] = data['total_income'].astype('int')  # Округлим до целого для удобства

Выводы:

1. Берем значения столбцов "Трудовой стаж" и "Зарплата" по модулю.
2. Заполняем пропуски NaN в столбцах "трудовой стаж", "зарплата": а. 'days_employed': NaN заменяем на произведение среднего коэффициента по столбцу 'income_type', умноженного на возраст в днях; б. 'total_income': NaN заменяем на среднее значение по зарплате по группе 'income_type'.

### Замена типа данных

In [48]:
### Обработка дубликатов

In [49]:
data.duplicated().sum() #Посчитаем количество дубликатов

71

In [50]:
data = data.drop_duplicates()
data.duplicated().sum() #Удаляем дубликаты и снова проверяем их наличие

0

### Вывод

### Лемматизация

In [51]:
unique_purposes = data['purpose'].value_counts().index.tolist()
print(data['purpose'].value_counts().to_frame())

                                        purpose
свадьба                                     791
на проведение свадьбы                       768
сыграть свадьбу                             765
операции с недвижимостью                    675
покупка коммерческой недвижимости           661
операции с жильем                           652
покупка жилья для сдачи                     651
операции с коммерческой недвижимостью       650
жилье                                       646
покупка жилья                               646
покупка жилья для семьи                     638
строительство собственной недвижимости      635
недвижимость                                633
операции со своей недвижимостью             627
строительство жилой недвижимости            624
покупка недвижимости                        621
покупка своего жилья                        620
строительство недвижимости                  619
ремонт жилью                                607
покупка жилой недвижимости              

In [52]:
#Лемматизируем столбец 'purpose' (цель получения кредита). Получим список уникальных лемм

In [53]:
m = Mystem()
list_of_lemmas = []
for element in data['purpose']:
    lemma = m.lemmatize(element)
    list_of_lemmas.extend(lemma)

unique_lemmas = Counter(list_of_lemmas)
sorted(unique_lemmas.items(), key = lambda pair: pair[1], reverse=True)

[(' ', 33570),
 ('\n', 21454),
 ('недвижимость', 6351),
 ('покупка', 5897),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2324),
 ('свой', 2230),
 ('на', 2222),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904),
 ('проведение', 768),
 ('сыграть', 765),
 ('сдача', 651),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('подержанный', 486),
 ('подержать', 478),
 ('приобретение', 461),
 ('профильный', 436)]

In [54]:
final_list_of_purposes = [k for k in sorted(unique_lemmas, key=unique_lemmas.get, reverse=True)
                          if len(k) > 4 if k != ' ' if k != '\n'][0:7]
print(final_list_of_purposes)

['недвижимость', 'покупка', 'жилье', 'автомобиль', 'образование', 'операция', 'свадьба']


In [55]:
'''Возьмем слова длиной от 4 до 7 букв, чтобы избавиться от лишнего мусораи оставить только существительные. 
В этом списке слова "покупка" и "операция" нам не нужны, т.к. относятся к жилью. Мы можем удалить их, 
'''

'Возьмем слова длиной от 4 до 7 букв, чтобы избавиться от лишнего мусораи оставить только существительные. \nВ этом списке слова "покупка" и "операция" нам не нужны, т.к. относятся к жилью. Мы можем удалить их, \n'

In [56]:
final_list_of_purposes.remove('покупка')
final_list_of_purposes.remove('операция')
print(final_list_of_purposes)

['недвижимость', 'жилье', 'автомобиль', 'образование', 'свадьба']


In [57]:
dict_purposes = {i:j for j,i in enumerate(final_list_of_purposes)}
dict_purposes['недвижимость'] = 1
print(dict_purposes)

{'недвижимость': 1, 'жилье': 1, 'автомобиль': 2, 'образование': 3, 'свадьба': 4}


In [58]:
'''Преобразуем лист в словарь, чтоб присваивать строкам категории:
1 - приобретение недвижимости/жилья (присвоим этим двум словам одну и ту же категорию);
2 - приобретение автомобиля;
3 - образование;
4 - свадьба.'''

'Преобразуем лист в словарь, чтоб присваивать строкам категории:\n1 - приобретение недвижимости/жилья (присвоим этим двум словам одну и ту же категорию);\n2 - приобретение автомобиля;\n3 - образование;\n4 - свадьба.'

### Вывод

### Категоризация данных

Надо получить основы слов для корректного присвоения категории

In [59]:
rus_stemm = SnowballStemmer('russian')

stem_purposes = [rus_stemm.stem(word) for word in final_list_of_purposes]
print(stem_purposes)

['недвижим', 'жил', 'автомобил', 'образован', 'свадьб']


Оформляем список целей ввиде словаря

In [61]:
dict_stemmed = {i:j for j,i in enumerate(stem_purposes)}
dict_stemmed['недвижим'] = 1
print(dict_stemmed)

{'недвижим': 1, 'жил': 1, 'автомобил': 2, 'образован': 3, 'свадьб': 4}


Присоединяем новый столбец в котором укажем категорию цели получения кредита.

In [75]:
data['purpose_cat'] = 0
for row in range(len(data)):
    for purpose in dict_stemmed:
        if purpose in data.iloc[row, 11]:
            data.iloc[row, 12] = dict_stemmed[purpose]
        
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_cat,income_cat,days_employed_cat,dob_years_cat,children_cat
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,богатый,стаж более 30 лет,30-45 лет,1-2 ребенка
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2,средний,стаж 10-30 лет,30-45 лет,1-2 ребенка
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1,богатый,стаж 10-30 лет,30-45 лет,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3,богатый,стаж 10-30 лет,30-45 лет,многодетные
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,4,богатый,стаж более 30 лет,45-65 лет,нет детей
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,1,богатый,стаж до 10 лет,до 30 лет,нет детей
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,1,богатый,стаж до 10 лет,30-45 лет,нет детей
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,3,богатый,стаж до 10 лет,45-65 лет,нет детей
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,4,средний,стаж более 30 лет,30-45 лет,1-2 ребенка
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,1,богатый,стаж до 10 лет,30-45 лет,нет детей


In [63]:
'''Функции для разбиения на разные классы'''

def income_cat(row): #по зарплате
    if row['total_income'] <= 50000:
        return 'бедный'
    elif 50000 < row['total_income'] <= 120000:
        return 'средний'
    elif 120000 < row['total_income'] < 1000000:
        return 'богатый'
    else:
        return 'миллионер'

    
def days_employed_cat(row): #по стажу
    if row['days_employed'] <= 3652:
        return 'стаж до 10 лет'
    elif 3652 < row['days_employed'] <= 6904:
        return 'стаж 10-30 лет'
    else:
        return 'стаж более 30 лет'


def dob_years_cat(row): #по возрасту
    if row['dob_years'] < 30:
        return 'до 30 лет'
    elif 30 <= row['dob_years'] < 45:
        return '30-45 лет'
    elif 45 <= row['dob_years'] < 65:
        return '45-65 лет'
    else:
        return 'старше 65 лет'

    
def children_cat(row): #по количеству детей
    if row['children'] == 0:
        return 'нет детей'
    elif 1 <= row['children'] <= 2:
        return '1-2 ребенка'
    else:
        return 'многодетные'

In [64]:
data['income_cat'] = data.apply(income_cat, axis=1)
data['days_employed_cat'] = data.apply(days_employed_cat, axis=1)
data['dob_years_cat'] = data.apply(dob_years_cat, axis=1)
data['children_cat'] = data.apply(children_cat, axis=1)

In [73]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_cat,income_cat,days_employed_cat,dob_years_cat,children_cat
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,богатый,стаж более 30 лет,30-45 лет,1-2 ребенка
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2,средний,стаж 10-30 лет,30-45 лет,1-2 ребенка
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1,богатый,стаж 10-30 лет,30-45 лет,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3,богатый,стаж 10-30 лет,30-45 лет,многодетные
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,4,богатый,стаж более 30 лет,45-65 лет,нет детей


In [74]:
### Код ревьювера ###

data['total_income'].quantile([0.25, 0.5, 0.75]) # 50-ый процентиль == медиана 

0.25    107623.0
0.50    151887.0
0.75    202417.0
Name: total_income, dtype: float64

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [67]:
def relation(category):
    return data.groupby(category)['debt'].mean().to_frame().sort_values(by='debt')

In [68]:
relation('children_cat')

,debt
children_cat,
нет детей,0.075438
многодетные,0.081579
1-2 ребенка,0.092654


### Вывод

Чем больше детей, тем больше вероятность просрочки. Клиенты без детей являются более свободны в распределении своих финансов. Также они более мобильны как в выборе профессии, так и в местонахождении работы (им легче сорваться с одного места в поиске высокооплачиваемой работы).

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [69]:
relation('family_status')

,debt
family_status,
вдовец / вдова,0.065693
в разводе,0.071130
женат / замужем,0.075452
гражданский брак,0.093471
Не женат / не замужем,0.097509


### Вывод

Люди, не зарегистрировавшие своих отношений, более легкомыслены в оплате кредитов. Человек, связанный узами (либо прошедший через это), ответственно относится к благосостоянию своей семьи, привык планировать будущее. Он, как правило, умеет распределять деньги, в том числе и на оплату кредитов. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [70]:
relation('income_cat')

,debt
income_cat,
бедный,0.061828
миллионер,0.080000
богатый,0.081459
средний,0.081570


### Вывод

Люди с маленьким доходом более ответственно подходят к распределению финансов и оплате долгов. Не богатые, как и миллионеры, точно знают зачем они берут кредит, для каких целей. И, чаще всего, продумали, как будут этот кредит оплачивать
Люди со средним достатком и доходом выше среднего чаще покупают какие-либо вещи (дорогие и не очень) просто потому, что "захотелось".

- Как разные цели кредита влияют на его возврат в срок?

Недвижимость - 1
Автомобиль   - 2
Образование  - 3
Свадьба      - 4

In [72]:
relation('purpose_cat')

,debt
purpose_cat,
1,0.072334
4,0.080034
3,0.092200
2,0.093590


### Вывод

Ипотечное рабство сильнее мотивирует людей на выплаты кредитов. Покупка своего дома процесс не быстрый и очень ответственный. К такому шагу подходят не спеша, обдумав все шаги, в том числе и оплату кредитов. 
По статистике автомобиль самый опасный вид транспорта. Поэтому не удивительно, что это самый "невозвращаемый" вид кредита. 
Далее, образование - если деньги берут родители для обучения своих детей это одно. Но если человек берет кредит на свое образование, с надеждой на увеличение дохода, то после обучения он может столкнуться с проблемой трудоустройства (как специалист без опыта работы).

### Шаг 4. Общий вывод

1. Самые лучшие клиенты: состоящие в законном браке и не имеющие детей
2. Самые худшие: Многодетные одиночки.
3. Лучше всего давать кредит на покупку жилья
4. Автомобиль и образование напротив худшее вложение ))